In [1]:
import torch
from src.transformer.wan.recam.model import WanRecamTransformer3DModel
import json
from accelerate import init_empty_weights
from glob import glob
from safetensors.torch import load_file

checkpoint = torch.load("/mnt/localssd/apex-diffusion/components/KwaiVGI/ReCamMaster-Wan2.1/resolve/main/step20000.ckpt")


In [ ]:

config = json.load(open("/mnt/localssd/apex-diffusion/components/Wan-AI_Wan2.1-T2V-1.3B-Diffusers/transformer/config.json"))

with init_empty_weights():
    model = WanRecamTransformer3DModel.from_config(config)

state_dict_files = glob('/mnt/localssd/apex-diffusion/components/Wan-AI_Wan2.1-T2V-1.3B-Diffusers/transformer/*.safetensors')
state_dict = checkpoint

for file in state_dict_files:
    state_dict.update(load_file(file, device="cpu"))

model.load_state_dict(state_dict,assign=True)
model.to("cuda")
model.eval()



In [ ]:
chec

In [ ]:
print(model.patch_embedding.weight.dtype)

In [ ]:
import sys 
import torch
import torch.nn as nn
sys.path.append("/home/tosinkuye/apex/ReCamMaster")
from diffsynth import ModelManager, WanVideoReCamMasterPipeline, save_video, VideoData

model_manager = ModelManager(torch_dtype=torch.bfloat16, device="cpu")
model_manager.load_models([
    "/mnt/localssd/models/Wan-AI/Wan2.1-T2V-1.3B/diffusion_pytorch_model.safetensors",
], torch_dtype=torch.float32)
pipe = WanVideoReCamMasterPipeline.from_model_manager(model_manager, device="cuda")

dit = pipe.dit

dim=dit.blocks[0].self_attn.q.weight.shape[0]
for block in dit.blocks:
    block.cam_encoder = nn.Linear(12, dim)
    block.projector = nn.Linear(dim, dim)
    block.cam_encoder.weight.data.zero_()
    block.cam_encoder.bias.data.zero_()
    block.projector.weight = nn.Parameter(torch.eye(dim))
    block.projector.bias = nn.Parameter(torch.zeros(dim))
# 3. Load ReCamMaster checkpoint
dit.load_state_dict(checkpoint, strict=True)

print(dit)

In [ ]:
m = torch.load('/home/tosinkuye/apex/ReCamMaster/latents_input.pt')

latents_input = m['latents_input'].to(torch.float32)
timestep = m['timestep'].to(torch.float32)
cam_emb = m['cam_emb'].to(torch.float32)
prompt_emb_posi = m['prompt_emb_posi'].to(torch.float32)
prompt_emb_nega = m['prompt_emb_nega'].to(torch.float32)

In [ ]:
with torch.no_grad():
    outs_a = model.forward(hidden_states=latents_input, timestep=timestep, encoder_hidden_states=prompt_emb_posi, cam_hidden_states=cam_emb)
    outs_b = dit.forward(latents_input, timestep, prompt_emb_posi, cam_emb)